In [1]:
from __future__ import division
import pandas as pd
import gensim
import gc
import generate_model

import evaluation
from gensim.models import word2vec

from concatenate_models import *

file_path = "type_sentences.txt"
load_file_with_types = "~/Scaricati/file.stdout"



model_e = generate_model.GenerateModel("../data/starting_data/entity_sentences.txt")
model_e = model_e.fit("entity_vectors", _window=3, _size=100, _min_count=5, _sg=0)
model_t = generate_model.GenerateModel("../data/starting_data/type_sentences.txt")
model_t = model_t.fit("type_vectors", _window=2, _size=25, _min_count=5, _sg=0)

print "Concatenating Models"
    # Concatenate Both Models
ct = ConcatenateEntityToType(_entity_model=model_e,
				 _type_model=model_t,
				 _types_file= "../data/starting_data/entity_to_type",
				 _total_size = 125)
model = ct.concatenate()

Using TensorFlow backend.


In [5]:
df = pd.read_csv("../data/starting_data/gold_standard", header=None, delimiter=r"\s+", names = ["First", "Second", "Third", "Fourth"])

In [7]:
with open("../data/evaluation_results/analyzing_results", "w") as f:
    correct_type = 0
    correct_entity = 0
    total_number = 0
    for index, row in df.iterrows():
        #print row['First'], row['Second']
        try:
            second = row['Second'].decode('utf-8')
            first = row['First'].decode('utf-8')
            third = row['Third'].decode('utf-8')
            fourth = row['Fourth'].decode('utf-8')
            predicted = model.wv.most_similar_cosmul(positive=[second, third], negative=[first])[0][0]
            predicted_entity = model_e.wv.most_similar_cosmul(positive=[row['Second'], row['Third']], negative=[row['First']])[0][0]

            if (predicted != fourth and predicted_entity != row['Fourth']):
                f.write("[BOTH WRONG] both wrong " + first + " " + second + " " + third + " "
                        + fourth + " type: " + predicted + " entity: " + predicted_entity + "\n")
            elif(predicted == fourth and predicted_entity != row['Fourth']):
                f.write("[TYPE] type ok " + first + " " + second + " " + third + " "
                        + fourth + " type: " + predicted + " entity: " + predicted_entity+ "\n")
                correct_type = correct_type + 1
            elif(predicted_entity == row['Fourth'] and predicted != fourth):
                f.write("[ENTITY] entity ok " + first + " " + second + " " + third + " "
                        + fourth + " type: " + predicted + " entity: " + predicted_entity+ "\n")
                correct_entity = correct_entity + 1
            else:
                f.write("[BOTH OK] OK " + first + " " + second + " " + third + " "
                        + fourth + " type: " + predicted + " entity: " + predicted_entity+ "\n")
                correct_type = correct_type + 1
                correct_entity = correct_entity + 1

            f.flush()

                #print "uncorrect", first, second, third, fourth, predicted
            total_number = total_number + 1
            if total_number%100 == 0:
                print correct_type / total_number, correct_entity / total_number, total_number
        except Exception as e:
            continue

print "FINAL score type", correct_type / total_number, "on", total_number
print "FINAL score entity", correct_entity / total_number, "on", total_number